# HW7: Neural Transition-Based Dependency Parsing


In this exercise, you are going to build a deep learning model for Neural Networks Transition-Based Dependency Parsing. A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between “head” words and words which modify those heads. Your implementation will be a transition-based parser, which incrementally builds up a parse one step at a time.

To complete this exercise, you will need to complete the code and build a deep learning model for dependency parsing. We will evaluate the model on the subset of Penn Treebank (annotated with Universal Dependencies). 

We provide the code for data preparation and the skeleton for PartialParse class. You do not need to understand the code outside of this notebook. 

This homework and the starter codes are adopt from Stanford University class CS224n.

This homework does not require you to use Google Cloud as the model is quite small (but you can still use it if you want)

#### Don't forget to shut down your instance on Gcloud when you are not using it

## 1. Transition-Based Dependency Parsing

Your implementation will be a transition-based parser, which incrementally builds
up a parse one step at a time. At every step it maintains a partial parse, which is represented as follows:
- A stack of words that are currently being processed.
- A buffer of words yet to be processed.
- A list of dependencies predicted by the parser.

Initially, the stack only contains ROOT, the dependencies lists is empty, and the buffer contains all words
of the sentence in order. At each step, the parse applies a transition to the partial parse until its buffer is
empty and the stack is size 1. The following transitions can be applied:
- SHIFT: removes the first word from the buffer and pushes it onto the stack.
- LEFT-ARC: marks the second (second most recently added) item on the stack as a dependent of the
first item and removes the second item from the stack.
- RIGHT-ARC: marks the first (most recently added) item on the stack as a dependent of the second
item and removes the first item from the stack.

Your parser will decide among transitions at each state using a neural network classifier.

<img src="images/img1.jpg">

### TODO 1 (Written):
Go through the sequence of transitions needed for parsing the sentence “I parsed
this sentence correctly”. The dependency tree for the sentence is shown below. At each step, give the
configuration of the stack and buffer, as well as what transition was applied this step and what new
dependency was added (if any). The first three steps are provided below as an example.

Complete the following table (double click the table and fill in the rest):

| stack    |  buffer |  new dependency | transition |
| :------: |:------: | :-------------: | :--------: |
| \[ROOT\]            | \[I, parsed, this, sentence, correctly\] |          | Initial Configuration |
| \[ROOT, I\]         | \[parsed, this, sentence, correctly\]    |          | SHIFT |
| \[ROOT, I, parsed\] | \[this, sentence, correctly\]            |          | SHIFT |
| \[ROOT, parsed\]    | \[this, sentence, correctly\]            | parsed→I | LEFT-ARC |
| \[ROOT, parsed, this\]    | \[ sentence, correctly\]           |          | SHIFT |
| \[ROOT, parsed, this, sentence\]    | \[ correctly\]           |          | SHIFT |
| \[ROOT, parsed, sentence\]    | \[correctly\]            | sentence→this | LEFT-ARC |
| \[ROOT, parsed\]    | \[correctly\]            | parsed→sentence | RIGHT-ARC |
| \[ROOT, parsed, correctly\] | \[\]            |          | SHIFT |
| \[ROOT, parsed\]    | \[\]            | parsed→correctly | RIGHT-ARC |
| \[ROOT\]    | \[\]            | ROOT→parsed | RIGHT-ARC |

### TODO 2 (Coding):
Implement the __\_\_init\_\___ and __parse_step__ functions in the PartialParse class. Your code must past both of the following tests.

In [1]:
class PartialParse(object):
    def __init__(self, sentence):
        """Initializes this partial parse.

        Your code should initialize the following fields:
            self.stack: The current stack represented as a list with the top of the stack as the
                        last element of the list.
            self.buffer: The current buffer represented as a list with the first item on the
                         buffer as the first item of the list
            self.dependencies: The list of dependencies produced so far. Represented as a list of
                    tuples where each tuple is of the form (head, dependent).
                    Order for this list doesn't matter.

        The root token should be represented with the string "ROOT"

        Args:
            sentence: The sentence to be parsed as a list of words.
                      Your code should not modify the sentence.
        """
        # The sentence being parsed is kept for bookkeeping purposes. Do not use it in your code.
        self.sentence = sentence

        ### YOUR CODE HERE
        self.stack = ['ROOT']
        self.buffer = sentence
        self.dependencies = []
        ### END YOUR CODE

    def parse_step(self, transition):
        """Performs a single parse step by applying the given transition to this partial parse

        Args:
            transition: A string that equals "S", "LA", or "RA" representing the shift, left-arc,
                        and right-arc transitions. You can assume the provided transition is a legal
                        transition.
        """
        ### YOUR CODE HERE
        if transition == 'S':
            self.stack = self.stack + [self.buffer[0]]
            self.buffer = self.buffer[1:]
        elif transition == 'LA':
            self.dependencies = self.dependencies+[(self.stack[-1],self.stack[-2])]
            self.stack = self.stack[:-2]+[self.stack[-1]]
        elif transition == 'RA':
            self.dependencies = self.dependencies+[(self.stack[-2],self.stack[-1])]
            self.stack = self.stack[:-1]
        ### END YOUR CODE

    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse

        Args:
            transitions: The list of transitions in the order they should be applied
        Returns:
            dependencies: The list of dependencies produced when parsing the sentence. Represented
                          as a list of tuples where each tuple is of the form (head, dependent)
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies

In [2]:
# Do not modify this code
def test_step(name, transition, stack, buf, deps,
              ex_stack, ex_buf, ex_deps):
    """Tests that a single parse step returns the expected output"""
    pp = PartialParse([])
    pp.stack, pp.buffer, pp.dependencies = stack, buf, deps

    pp.parse_step(transition)
    stack, buf, deps = (tuple(pp.stack), tuple(pp.buffer), tuple(sorted(pp.dependencies)))
    assert stack == ex_stack, \
        "{:} test resulted in stack {:}, expected {:}".format(name, stack, ex_stack)
    assert buf == ex_buf, \
        "{:} test resulted in buffer {:}, expected {:}".format(name, buf, ex_buf)
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)
    print("{:} test passed!".format(name))


def test_parse_step():
    """Simple tests for the PartialParse.parse_step function
    Warning: these are not exhaustive
    """
    test_step("SHIFT", "S", ["ROOT", "the"], ["cat", "sat"], [],
              ("ROOT", "the", "cat"), ("sat",), ())
    test_step("LEFT-ARC", "LA", ["ROOT", "the", "cat"], ["sat"], [],
              ("ROOT", "cat",), ("sat",), (("cat", "the"),))
    test_step("RIGHT-ARC", "RA", ["ROOT", "run", "fast"], [], [],
              ("ROOT", "run",), (), (("run", "fast"),))


def test_parse():
    """Simple tests for the PartialParse.parse function
    Warning: these are not exhaustive
    """
    sentence = ["parse", "this", "sentence"]
    dependencies = PartialParse(sentence).parse(["S", "S", "S", "LA", "RA", "RA"])
    dependencies = tuple(sorted(dependencies))
    expected = (('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
    assert dependencies == expected,  \
        "parse test resulted in dependencies {:}, expected {:}".format(dependencies, expected)
    assert tuple(sentence) == ("parse", "this", "sentence"), \
        "parse test failed: the input sentence should not be modified"
    print("parse test passed!")

In [3]:
test_parse_step()
test_parse()

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


### TODO 3 (Coding):
Our network will predict which transition should be applied next to a partial parse. We could use it to parse a single sentence by applying predicted transitions until the parse is complete. However, neural networks run much more efficiently when making predictions about batches of data at a time (i.e., predicting the next transition for a many different partial parses simultaneously). We can parse sentences in minibatches with the following algorithm.

<img src="images/img2.jpg">

Implement this algorithm in the minibatch parse function and run the following test.

In [4]:
import numpy as np
def minibatch_parse(sentences, model, batch_size):
    """Parses a list of sentences in minibatches using a model.

    Args:
        sentences: A list of sentences to be parsed (each sentence is a list of words)
        model: The model that makes parsing decisions. It is assumed to have a function
               model.predict(partial_parses) that takes in a list of PartialParses as input and
               returns a list of transitions predicted for each parse. That is, after calling
                   transitions = model.predict(partial_parses)
               transitions[i] will be the next transition to apply to partial_parses[i].
        batch_size: The number of PartialParses to include in each minibatch
    Returns:
        dependencies: A list where each element is the dependencies list for a parsed sentence.
                      Ordering should be the same as in sentences (i.e., dependencies[i] should
                      contain the parse for sentences[i]).
    """

    ### YOUR CODE HERE
    partial_parses = [PartialParse(i) for i in sentences]
    dependencies = []
    unfinished_parses = [partial_parses[i:i+batch_size] for i in range(0,len(partial_parses),batch_size)]
    for minibatch in unfinished_parses:
        
        enum_minibatch = np.array([[i,j] for i,j in enumerate(minibatch)])
        tmp = [[]]*batch_size

        while(enum_minibatch.size > 0):
            transitions = model.predict(enum_minibatch[:,1])
            tmp_enum = enum_minibatch
            for i in range(len(enum_minibatch)):
                if not (len(enum_minibatch[i][1].stack) == 1 and len(enum_minibatch[i][1].buffer) == 0): 
                    enum_minibatch[i][1].parse_step(transitions[i]) 
 
                if len(enum_minibatch[i][1].stack) == 1 and len(enum_minibatch[i][1].buffer) == 0:  
                    tmp[enum_minibatch[i][0]] = enum_minibatch[i][1].dependencies
                    tmp_enum = np.concatenate((tmp_enum[:i],tmp_enum[i+1:]),axis=0)
                    
            enum_minibatch = tmp_enum

        dependencies += (tmp)
        del tmp

    ### END YOUR CODE

    return dependencies

In [5]:
# Do not modify this code
class DummyModel(object):
    """Dummy model for testing the minibatch_parse function
    First shifts everything onto the stack and then does exclusively right arcs if the first word of
    the sentence is "right", "left" if otherwise.
    """
    def predict(self, partial_parses):
        return [("RA" if pp.stack[1] is "right" else "LA") if len(pp.buffer) == 0 else "S"
                for pp in partial_parses]


def test_dependencies(name, deps, ex_deps):
    """Tests the provided dependencies match the expected dependencies"""
    deps = tuple(sorted(deps))
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)


def test_minibatch_parse():
    """Simple tests for the minibatch_parse function
    Warning: these are not exhaustive
    """
    sentences = [["right", "arcs", "only"],
                 ["right", "arcs", "only", "again"],
                 ["left", "arcs", "only"],
                 ["left", "arcs", "only", "again"]]
    deps = minibatch_parse(sentences, DummyModel(), 2)
    test_dependencies("minibatch_parse", deps[0],
                      (('ROOT', 'right'), ('arcs', 'only'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[1],
                      (('ROOT', 'right'), ('arcs', 'only'), ('only', 'again'), ('right', 'arcs')))
    test_dependencies("minibatch_parse", deps[2],
                      (('only', 'ROOT'), ('only', 'arcs'), ('only', 'left')))
    test_dependencies("minibatch_parse", deps[3],
                      (('again', 'ROOT'), ('again', 'arcs'), ('again', 'left'), ('again', 'only')))
    print("minibatch_parse test passed!")

In [6]:
test_minibatch_parse()

minibatch_parse test passed!


## 2. Setup and Preprocessing

In [7]:
from utils.parser_utils import minibatches, load_and_preprocess_data

Preparing data. We will use a subset of Penn Treebank and pretrained embeddings in this task

We are now going to train a neural network to predict, given the state of the stack, buffer, and dependencies, which transition should be applied next. First, the model extracts a feature vector representing the current state. We will be using the feature set presented in the original neural dependency parsing paper: A Fast and Accurate Dependency Parser using Neural Networks. 

The function extracting these features has been implemented for you in parser_utils. This feature vector consists of a list of tokens (e.g., the last word in the stack, first word in the buffer, dependent of the second-to-last word in the stack if there is one, etc.). They can be represented as a list of integers.

In [8]:
parser, embeddings, train_examples, dev_set, test_set = load_and_preprocess_data(True)

Loading data...
took 1.58 seconds
Building parser...
took 0.03 seconds
Loading pretrained embeddings...
took 1.64 seconds
Vectorizing data...
took 0.05 seconds
Preprocessing training data...
took 0.83 seconds


In [9]:
print(len(train_examples), len(dev_set), len(test_set))

48390 500 500


In [10]:
print(embeddings.shape)

(5157, 50)


Get the full batch of our subset data

In [11]:
minibatch_gen = minibatches(train_examples, len(train_examples))
x_train, y_train = minibatch_gen.__next__()

In [12]:
print(x_train.shape)
print(y_train.shape)

(48390, 36)
(48390, 3)


In [13]:
## Sample features
print(x_train[10])
print(y_train[10])

[5155 5156  621 5155 5155 5155  329   87  149  486  165 5155 5155 5155
 5155 5155 5155 5155   83   84   53   83   83   83   47   46   39   44
   47   83   83   83   83   83   83   83]
[0. 1. 0.]


### 3. Model

In [14]:
from keras.models import Sequential, Model
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Reshape, Dropout, Flatten
from keras.optimizers import Adam

Using TensorFlow backend.


### TODO 4 (Coding):
Build and train a keras model to predict an action for each state of of the input. This is a simple classification task. 
- The input and output of the model must match the dimention of x_train and y_train.
- The model must use the provided pretrained embeddings
- The model could comprise of only a feedforward layer and a dropout
- Training loss should be around 0.1 or below, and training categorical_accuracy above 0.94

In [15]:
from keras.layers import Conv1D, MaxPooling1D, TimeDistributed
from keras.metrics import categorical_accuracy
def get_your_object():
    max_features = embeddings.shape[0]
    max_len = x_train.shape[1]
    #replace "pass" with code for your neural net
    input1 = Input(shape=(max_len,))
    x = Embedding(max_features, 50,weights=[embeddings], input_length=max_len,trainable=True)(input1)
    #x = Conv1D(100, 5, strides = 1, padding='same', activation='relu')(x)
    #x = TimeDistributed(Dense(5, activation='relu'))(x)
    #x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(.5)(x)
    x = Dense(100, activation='relu')(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    out = Dense((y_train.shape[1]), activation='softmax')(x)

    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                loss='categorical_crossentropy',
                metrics=[categorical_accuracy])
    return model

obj_model = get_your_object()
obj_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 36)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 36, 50)            257850    
_________________________________________________________________
dense_1 (Dense)              (None, 36, 100)           5100      
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 100)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 36, 100)           10100     
_________________________________________________________________
flatten_1 (Flatten)  

In [16]:
%%time
obj_model.fit(x_train,y_train,batch_size=16,epochs=10,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
48390/48390 [==============================] - 38s 777us/step - loss: 0.3147 - categorical_accuracy: 0.8720
Epoch 2/10
48390/48390 [==============================] - 34s 711us/step - loss: 0.1894 - categorical_accuracy: 0.9236
Epoch 3/10
48390/48390 [==============================] - 34s 707us/step - loss: 0.1519 - categorical_accuracy: 0.9399
Epoch 4/10
48390/48390 [==============================] - 33s 691us/step - loss: 0.1278 - categorical_accuracy: 0.9494
Epoch 5/10
48390/48390 [==============================] - 36s 742us/step - loss: 0.1102 - categorical_accuracy: 0.9586
Epoch 6/10
48390/48390 [==============================] - 29s 590us/step - loss: 0.0950 - categorical_accuracy: 0.9635
Epoch 7/10
48390/48390 [==============================] - 29s 597us/step - loss: 0.0832 - categorical_accuracy: 0.9676
Epoch 8/10
48390/48390 [==============================] - 32s 670us/step - loss: 0.0724 - categorical_accuracy: 0.9725


## 4. Evaluation

For Dependency Parsing, we usually report attachment score of the model for evaluation. There are two possible metrics UAS and LAS.

### TODO 5 (Written and Coding):
Explain how attachment score is calculated and the difference between unlabeled attachment score (UAS) and labeled attachment score (LAS). Which one should we use here?

__Answer here__:

Attachment score is the percentage of words that have the correct head.

UAS studies the structure of a dependency tree and assesses whether the output has the correct
head and dependency arcs. In addition to the structure score in UAS, LAS also measures the accuracy
of the dependency labels on each arc. A third, but less common metric, is used to judge the percentage
of sentences that are completely correct in regards to their LAS score. This score is better used to judge
how a dependency parser will affect other NLP tools that make use of the dependency parser output.

but this homework will use only UAS.

Report the score using appropriate metric on dev_set and test_set. The function for calculating scores are provided in parser.parse and the dataset can be passed in as-is.

dev_score:

In [17]:
UAS, depen = parser.parse(dev_set, obj_model, minibatch_parse, eval_batch_size=256)
UAS

0.1588769474298092

test_score:

In [18]:
UAS, depen = parser.parse(test_set, obj_model, minibatch_parse, eval_batch_size=256)
UAS

0.20168208578637511

Also, print one sample sentence (in English) in the test set and its predicted dependencies from the model.
You can use __parser.id2tok\[word_id\]__ to lookup the word in English.

__Draw a picture of this sentence with arrows and upload it to my couseville__

In [19]:
i = 20
print(test_set[i]['word'])
print(test_set[i]['pos'])
print(test_set[i]['head'])
print(test_set[i]['label'])
print('Word =',''.join([parser.id2tok[i]+'  ' for i in test_set[i]['word']]))
print('PoS =',''.join([parser.id2tok[i]+'  ' for i in test_set[i]['pos']]))
depen[i]

[5156, 85, 639, 767, 5154, 98, 105, 841, 391, 87]
[84, 41, 42, 48, 49, 39, 40, 42, 42, 46]
[-1, 2, 3, 0, 5, 3, 8, 8, 3, 3]
[-1, 27, 12, 0, 28, 6, 36, 17, 5, 20]
Word = <ROOT>  the  dow  fell  <UNK>  %  on  black  monday  .  
PoS = <p>:<ROOT>  <p>:DT  <p>:NNP  <p>:VBD  <p>:CD  <p>:NN  <p>:IN  <p>:NNP  <p>:NNP  <p>:.  


[(2, 1), (3, 2), (5, 4), (3, 5), (8, 7), (8, 6), (3, 8), (3, 9), (0, 3)]

In [21]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://doc-04-a8-docs.googleusercontent.com/docs/securesc/tqv3ev9luo67j4u7api4ed0bb4fuhn4j/lesukbum738ku5o5e2q17fkcdba7j1i2/1552744800000/01909380670798354122/01909380670798354122/1jLlMR9t2hHlJGe0O2hbQr_aHeol9OidU?e=download")

In [23]:
Image(url= "https://scontent.fbkk10-1.fna.fbcdn.net/v/t1.0-9/54520859_2050699871665657_8230014571658608640_o.jpg?_nc_cat=105&_nc_eui2=AeGm9Jq4IyspKkx69bpK9Uof7ya4ExEqc_KM2DIGJ0-GhlZCztchFymnagp3dbCZZp3ZQ-17K26QTRRcxAHro1vM3ZXj2bIfGNdahUH1W_F6bA&_nc_ht=scontent.fbkk10-1.fna&oh=ef457626a2798b1c6c86317af1fcd36c&oe=5D242710")

#### Don't forget to shut down your instance on Gcloud when you are not using it